## Photometry on MIRI images

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import subprocess
import miri_utils.photometry_tools as phot

from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.table import Table

warnings.simplefilter("ignore", category=FITSFixedWarning)


cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"


# Section to obtain modified apertures

Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
#print(table[:5])
table.info()
print(table.columns)

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = phot.adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_v2.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '10245'
filter = 'F770W'
survey_obs = 'primer004'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table['Image_Err'].shape)

# This is where we do most of the work

# Section to perform the actual photometry

First, let's set some parameters straight:

pixscale_arcsec = 0.11092  # arcsec per pixel

pix_area_sr = 2.89208962133982e-13  # from MIRI header


In case the code should be tested

In [ ]:
test_ids = ['8465', '7922', '9871', '12202', '8843', '7904', '8338', '10021', '10245', '11136', '12340', '20397']

In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table_small = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_small.csv'
aperture_table_big = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_big.csv'
#fig_path = '/home/bpc/University/master/Red_Cardinal/photometry/Plots_MIRI_phot_v3/'
fig_path = None
os.makedirs(output_folder, exist_ok=True)

# Get all possible F770W files
all_f770w_files = glob.glob(os.path.join(cutouts_folder, f'*F770W*.fits'))

# Group F770W files by galaxy ID and filter
f770w_files = []
galaxy_ids = set([os.path.basename(f).split('_')[0] for f in all_f770w_files])

for galaxy_id in galaxy_ids:
    # Find all F770W files for this galaxy ID
    matching_files = [f for f in all_f770w_files if os.path.basename(f).startswith(galaxy_id)]
    
    # Handle special case for galaxy 11853
    if galaxy_id == '11853':
        # Use the cweb2 file if available
        cweb2_files = [f for f in matching_files if 'cweb2' in f.lower()]
        if cweb2_files:
            f770w_files.append(cweb2_files[0])
            continue  # Skip to the next galaxy
    
    # Prioritise PRIMER over COSMOS-Web
    primer_files = [f for f in matching_files if 'primer' in f.lower()]
    cweb_files = [f for f in matching_files if 'cweb' in f.lower()]
    
    if primer_files:
        f770w_files.append(primer_files[0])  # Prefer PRIMER file
    elif cweb_files:
        f770w_files.append(cweb_files[0])  # Use CWEB only if no PRIMER available

# Get all F1800W files
f1800w_files = glob.glob(os.path.join(cutouts_folder, f'*F1800W*.fits'))

psf_f770w = phot.get_psf('F770W')
psf_f1800w = phot.get_psf('F1800W')

# Small aperture without aperture correction:

phot.perform_photometry(f770w_files, aperture_table_small, output_folder, 
                        psf_f770w, suffix='_small_no_corr',
                        sigma=2.0, annulus_factor=3.0
)

phot.perform_photometry(f1800w_files, aperture_table_small, output_folder, 
                        psf_f1800w, suffix='_small_no_corr',
                        sigma=2.0, annulus_factor=3.0
)

# Big Aperture without aperture correction:

phot.perform_photometry(f770w_files, aperture_table_big, output_folder, 
                        psf_f770w, suffix='_big_no_corr',
                        sigma=2.0, annulus_factor=3.0
)

phot.perform_photometry(f1800w_files, aperture_table_big, output_folder, 
                        psf_f1800w, suffix='_big_no_corr',
                        sigma=2.0, annulus_factor=3.0
)

# Plotting:

if fig_path:
    phot.combine_figures(fig_path)

Create the FITS table for small and big aperture

In [ ]:
# Small aperture

# Your existing code to save CSV files for each filter
fits_table_small = f'Flux_SmallAperture_NoCorr_MIRI.fits'
f770w_fname = 'phot_table_F770W_small_no_corr.csv'
f1800w_fname = 'phot_table_F1800W_small_no_corr.csv'

# Now create the combined FITS table
phot.combine_filter_csv_to_fits(f770w_fname, f1800w_fname, fits_table_name=fits_table_small)



# Big aperture

# Your existing code to save CSV files for each filter
fits_table_big = f'Flux_BigAperture_NoCorr_MIRI.fits'
f770w_fname = 'phot_table_F770W_big_no_corr.csv'
f1800w_fname = 'phot_table_F1800W_big_no_corr.csv'

# Now create the combined FITS table
phot.combine_filter_csv_to_fits(f770w_fname, f1800w_fname, fits_table_name=fits_table_big)


In [ ]:
table1_path =  '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v3.fits'

print("Smaller aperture:")
table = Table.read(table1_path)
row = table[table['ID']=='7934']
print(row)
row = table[table['ID']=='12202']
print(row)
row = table[table['ID']=='11136']
print(row)


print("\n")
print("Bigger aperture:")
table2_path = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v2.fits'
table = Table.read(table2_path)
row = table[table['ID']=='7934']
print(row)
row = table[table['ID']=='12202']
print(row)
row = table[table['ID']=='11136']
print(row)

Compare the two catalogues:

In [ ]:
# Load the two tables
table_small_path =  '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_SmallAperture_NoCorr_MIRI.fits'
table_big_path =  '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_BigAperture_NoCorr_MIRI.fits'


table_small = Table.read(table_small_path)
table_big = Table.read(table_big_path)

# Convert ID columns to string for alignment
ids1 = [id.decode() if isinstance(id, bytes) else str(id) for id in table_small['ID']]
ids2 = [id.decode() if isinstance(id, bytes) else str(id) for id in table_big['ID']]

# Match common IDs
common_ids = sorted(set(ids1) & set(ids2))
print(f"Found {len(common_ids)} common galaxies")

# Collect differences for each band (F770W = index 0, F1800W = index 1)
diffs = {'Flux': [], 'Apr_Corr': []}
bands = ['F770W', 'F1800W']

for idx in [0, 1]:
    flux_diffs = []
    corr_diffs = []
    
    for gid in common_ids:
        i1 = ids1.index(gid)
        i2 = ids2.index(gid)

        flux_small = table_small['Flux'][i1][idx]*1e6
        flux_big = table_big['Flux'][i2][idx]*1e6
        corr_small = table_small['Apr_Corr'][i1][idx] if 'Apr_Corr' in table_small.colnames else np.nan
        corr_big = table_big['Apr_Corr'][i2][idx] if 'Apr_Corr' in table_big.colnames else np.nan

        # Only compare valid fluxes
        if np.isfinite(flux_small) and np.isfinite(flux_big):
            flux_diffs.append(flux_big - flux_small)
        
        if np.isfinite(corr_small) and np.isfinite(corr_big):
            corr_diffs.append(corr_big - corr_small)
    
    diffs['Flux'].append(flux_diffs)
    diffs['Apr_Corr'].append(corr_diffs)

    # Print basic statistics
    print(f"\n📊 Band: {bands[idx]}")
    print(f"Flux difference: mean={np.mean(flux_diffs):.3e}, median={np.median(flux_diffs):.3e}, std={np.std(flux_diffs):.3e}")
    print(f"Aperture correction difference: mean={np.mean(corr_diffs):.3f}, median={np.median(corr_diffs):.3f}, std={np.std(corr_diffs):.3f}")


# --- Plotting ---
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
props = dict(boxstyle='round', facecolor='white', alpha=0.85)

for i, band in enumerate(bands):
    flux_diffs = diffs['Flux'][i]
    corr_diffs = diffs['Apr_Corr'][i]

    # Flux
    ax_flux = axs[i, 0]
    ax_flux.hist(flux_diffs, bins=30, alpha=0.7, color='cornflowerblue')
    ax_flux.set_title(f"{band} Flux Difference")
    ax_flux.set_xlabel("Flux_large_apr - Flux_small_apr [µJy]")

    text_flux = '\n'.join((
        f"Mean:   {np.mean(flux_diffs):.2f} µJy",
        f"Median: {np.median(flux_diffs):.2f} µJy",
        f"Std:    {np.std(flux_diffs):.2f} µJy"
    ))
    ax_flux.text(0.25, 0.95, text_flux, transform=ax_flux.transAxes,
                 fontsize=9, verticalalignment='top', horizontalalignment='right', bbox=props)

    # Aperture Correction
    ax_corr = axs[i, 1]
    ax_corr.hist(corr_diffs, bins=30, alpha=0.7, color='darkorange')
    ax_corr.set_title(f"{band} Aperture Correction Difference")
    ax_corr.set_xlabel("Corr_large_apr - Corr_small_apr")

    text_corr = '\n'.join((
        f"Mean:   {np.mean(corr_diffs):.2f}",
        f"Median: {np.median(corr_diffs):.2f}",
        f"Std:    {np.std(corr_diffs):.2f}"
    ))
    ax_corr.text(0.2, 0.95, text_corr, transform=ax_corr.transAxes,
                 fontsize=9, verticalalignment='top', horizontalalignment='right', bbox=props)

plt.suptitle('Comparison between fluxes using small and large aperture')
plt.tight_layout()
plt.show()



This instead is the CLAUDE version of the above analysis - as always completely overpowered:

In [ ]:
phot_dir = '/home/bpc/University/master/Red_Cardinal/photometry/'

table_small_path = os.path.join(phot_dir, 'aperture_table_small.csv')
table_big_path = os.path.join(phot_dir, 'aperture_table_big.csv')
fig_path = os.path.join(phot_dir, 'aperture_comparisons/aperture_statistics_log.png')
summary_doc_path = os.path.join(phot_dir, 'aperture_comparisons/summary_statistics.txt')

phot.compare_aperture_statistics(table_small_path, table_big_path, fig_path, summary_doc_path)

Collect galaxy IDs and their corresponding observations:

In [ ]:
NIRCam_table =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
table = Table.read(NIRCam_table)
table.info()
f444w_ids = list(table['ID'])

MIRI_table = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI.fits'
table = Table.read(MIRI_table)
table.info()
f770w_ids = list(table['ID'])

cutout_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_phot/'
f1800w_ids = glob.glob(os.path.join(cutout_dir, '*F1800W*.fits'))
for i, f in enumerate(f1800w_ids):
    f = os.path.basename(f).split('_')[0]
    f1800w_ids[i] = f

# Convert the arrays to sets for efficient comparison
set_f444w = set(f444w_ids)
set_f770w = set(f770w_ids)
set_f1800w = set(f1800w_ids)

# 1) IDs in all 3 arrays
in_all_three = set_f444w & set_f770w & set_f1800w

# 2) IDs in f444w_ids and f770w_ids only (not in f1800w_ids)
in_f444w_and_f770w_only = (set_f444w & set_f770w) - set_f1800w

# 3) IDs only in f444w_ids and not in the other 2
only_in_f444w = set_f444w - set_f770w - set_f1800w

# Convert back to sorted lists if needed
in_all_three = sorted(list(in_all_three))
in_f444w_and_f770w_only = sorted(list(in_f444w_and_f770w_only))
only_in_f444w = sorted(list(only_in_f444w))

print(f"{len(in_all_three)} IDs in all three arrays: {in_all_three}")
print(f"{len(in_f444w_and_f770w_only)} IDs in f444w_ids and f770w_ids only: {in_f444w_and_f770w_only}")
print(f"{len(only_in_f444w)} IDs only in f444w_ids: {only_in_f444w}")



Check the contents of the original FITS files:

In [ ]:
fname = './../cutouts/7102_F770W_cutout_cweb1.fits'
#with fits.open(fname) as hdul:
    #hdul.info()


fname = './../MIRI/PRIMER_003/jw01837-o003_t003_miri_f770w_i2d.fits'
with fits.open(fname) as hdul:
    hdul.info()

fname = './../MIRI_shifted/PRIMER_003_shifted/jw01837-o003_t003_miri_f770w_i2d_shifted.fits'
with fits.open(fname) as hdul:
    hdul.info()



# Section to compare my aperture photometry to the photometry of the COSMOS-Web2025 catalogue

In [ ]:
import numpy as np
from matplotlib import pylab as plt
from astropy.io import ascii, fits
from astropy.table import Table, join

Reduce the big table to the necessary columns:

In [ ]:
from astropy.table import Table

# Read full table once (on a powerful machine)
#full = Table.read("/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_mastercatalog_v1_photom_primary.fits")

# Extract only the needed columns
#small = full['id', 'ra', 'dec', 'flux_auto_f770w', 'flux_err_auto_f770w', 'mag_auto_f770w']

# Save to new FITS or CSV
#small.write("/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_reduced.fits", overwrite=True)


In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

cweb_path = '/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_reduced.fits'
my_path = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v3.fits'
cat_path = '/home/bpc/University/master/Red_Cardinal/catalogues/cat_targets.fits'

my_table = Table.read(my_path, hdu=1)
cosmos_table = Table.read(cweb_path, hdu=1)
my_cat = Table.read(cat_path, hdu=1)
fits.info(my_path)
fits.info(cweb_path)
#print(my_cat.columns)

# Rename ID column to id to match other catalogues
my_table.rename_column('ID', 'id')

# Reduce catalogue
my_cat_small = my_cat['id', 'ra', 'dec']

# Force type setting
my_table['id'] = my_table['id'].astype(str)
my_cat_small['id'] = my_cat_small['id'].astype(str)

# Match according to IDs
matched = join(my_table, my_cat_small, keys=('id'), join_type='inner')

print(matched.columns)

matched.write('/home/bpc/University/master/Red_Cardinal/catalogues/Flux_Aperture_PSFMatched_AperCorr_MIRI_ra_dec.fits')

In [ ]:
# Build coordinates
coords_my = SkyCoord(ra=my_cat['ra']*u.deg, dec=my_cat['dec']*u.deg)
coords_cosmos = SkyCoord(ra=cosmos_table['ra']*u.deg, dec=cosmos_table['dec']*u.deg)

# Match (within 0.3 arcsec, for instance)
idx, d2d, _ = coords_my.match_to_catalog_sky(coords_cosmos)
match_mask = d2d < 0.3 * u.arcsec

# Build matched table
my_matched = my_table[match_mask]
cosmos_matched = cosmos_table[idx[match_mask]]

# Combine tables: rename columns to avoid name collision
cosmos_matched.rename_columns(
    cosmos_matched.colnames,
    [name + "_cosmos" if name in my_matched.colnames else name for name in cosmos_matched.colnames]
)

# Stack into one comparison table
from astropy.table import hstack
#sky_matched = hstack([my_matched, cosmos_matched])

# Merge horizontally
sky_matched = hstack([my_matched, cosmos_matched])

sky_matched.write('/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_matched.fits')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import fits
import matplotlib.patches as patches
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def load_and_prepare_data(fits_file_path):
    """
    Load the FITS table and prepare data for comparison
    """
    # Load the FITS table
    table = Table.read(fits_file_path)
    
    # Extract F770W data from multi-dimensional arrays
    # For Flux and Flux_Err, take the first value (F770W filter)
    flux_personal = []
    flux_err_personal = []
    ab_mag_personal = []
    
    for i in range(len(table)):
        # Handle Flux (F770W is first element)
        if table['Flux'].mask[i][0] == False:
            flux_val = table['Flux'][i]*1e6 # convert to µJy
            if hasattr(flux_val, '__len__') and len(flux_val) > 0:
                flux_personal.append(flux_val[0])
            else:
                flux_personal.append(flux_val)
        else:
            flux_personal.append(np.nan)
            
        # Handle Flux_Err (F770W is first element)
        if table['Flux_Err'].mask[i][0] == False:
            flux_err_val = table['Flux_Err'][i]*1e6 # convert to µJy
            if hasattr(flux_err_val, '__len__') and len(flux_err_val) > 0:
                flux_err_personal.append(flux_err_val[0])
            else:
                flux_err_personal.append(flux_err_val)
        else:
            flux_err_personal.append(np.nan)
            
        # Handle AB_Mag (F770W is first element)
        if table['AB_Mag'].mask[i][0] == False:
            ab_mag_val = table['AB_Mag'][i]
            if hasattr(ab_mag_val, '__len__') and len(ab_mag_val) > 0:
                ab_mag_personal.append(ab_mag_val[0])
            else:
                ab_mag_personal.append(ab_mag_val)
        else:
            ab_mag_personal.append(np.nan)
    
    # Convert to numpy arrays
    flux_personal = np.array(flux_personal)
    flux_err_personal = np.array(flux_err_personal)
    ab_mag_personal = np.array(ab_mag_personal)
    
    # Extract public catalogue data
    flux_public = np.array(table['flux_auto_f770w'])
    flux_err_public = np.array(table['flux_err_auto_f770w'])
    mag_public = np.array(table['mag_auto_f770w'])
    
    # Create masks for valid data
    valid_flux = (~np.isnan(flux_personal)) & (~np.isnan(flux_public)) & \
                 (flux_personal > 0) & (flux_public > 0)
    valid_flux_err = (~np.isnan(flux_err_personal)) & (~np.isnan(flux_err_public)) & \
                     (flux_err_personal > 0) & (flux_err_public > 0)
    valid_mag = (~np.isnan(ab_mag_personal)) & (~np.isnan(mag_public))
    
    return {
        'table': table,
        'flux_personal': flux_personal,
        'flux_err_personal': flux_err_personal,
        'ab_mag_personal': ab_mag_personal,
        'flux_public': flux_public,
        'flux_err_public': flux_err_public,
        'mag_public': mag_public,
        'valid_flux': valid_flux,
        'valid_flux_err': valid_flux_err,
        'valid_mag': valid_mag
    }

def calculate_statistics(x, y, valid_mask):
    """
    Calculate comparison statistics
    """
    if np.sum(valid_mask) < 3:
        return {}
    
    x_valid = x[valid_mask]
    y_valid = y[valid_mask]
    
    # Linear correlation
    corr_coef, p_value = stats.pearsonr(x_valid, y_valid)
    
    # Calculate residuals and statistics
    residuals = y_valid - x_valid
    mean_residual = np.mean(residuals)
    std_residual = np.std(residuals)
    rms_residual = np.sqrt(np.mean(residuals**2))
    
    # Fractional differences for positive values
    frac_diff = (y_valid - x_valid) / x_valid
    mean_frac_diff = np.mean(frac_diff)
    std_frac_diff = np.std(frac_diff)
    
    return {
        'correlation': corr_coef,
        'p_value': p_value,
        'mean_residual': mean_residual,
        'std_residual': std_residual,
        'rms_residual': rms_residual,
        'mean_frac_diff': mean_frac_diff,
        'std_frac_diff': std_frac_diff,
        'n_objects': len(x_valid)
    }

def create_comparison_plot(data):
    """
    Create comprehensive comparison plots
    """
    fig = plt.figure(figsize=(20, 15))
    
    # Define color scheme
    colors = {'scatter': '#1f77b4', 'line': '#ff7f0e', 'hist': '#2ca02c'}
    
    # 1. Flux comparison (log-log scale)
    ax1 = plt.subplot(3, 4, 1)
    valid_flux = data['valid_flux']
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        
        ax1.scatter(x_flux, y_flux, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_flux), np.min(y_flux))
        max_val = max(np.max(x_flux), np.max(y_flux))
        ax1.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax1.set_xscale('log')
        ax1.set_yscale('log')
        ax1.set_xlabel('Personal Catalogue Flux (µJy)')
        ax1.set_ylabel('Public Catalogue Flux (µJy)')
        ax1.set_title('Flux Comparison (F770W)')
        ax1.grid(True, alpha=0.3)
        
        # Add statistics
        stats_flux = calculate_statistics(x_flux, y_flux, np.ones(len(x_flux), dtype=bool))
        if stats_flux:
            ax1.text(0.05, 0.95, f'r = {stats_flux["correlation"]:.3f}\nN = {stats_flux["n_objects"]}', 
                    transform=ax1.transAxes, verticalalignment='top', 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 2. Flux error comparison (log-log scale)
    ax2 = plt.subplot(3, 4, 2)
    valid_flux_err = data['valid_flux_err']
    if np.sum(valid_flux_err) > 0:
        x_err = data['flux_err_personal'][valid_flux_err]
        y_err = data['flux_err_public'][valid_flux_err]
        
        ax2.scatter(x_err, y_err, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_err), np.min(y_err))
        max_val = max(np.max(x_err), np.max(y_err))
        ax2.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax2.set_xscale('log')
        ax2.set_yscale('log')
        ax2.set_xlabel('Personal Catalogue Flux Error (µJy)')
        ax2.set_ylabel('Public Catalogue Flux Error (µJy)')
        ax2.set_title('Flux Error Comparison (F770W)')
        ax2.grid(True, alpha=0.3)
        
        # Add statistics
        stats_err = calculate_statistics(x_err, y_err, np.ones(len(x_err), dtype=bool))
        if stats_err:
            ax2.text(0.05, 0.95, f'r = {stats_err["correlation"]:.3f}\nN = {stats_err["n_objects"]}', 
                    transform=ax2.transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 3. AB Magnitude comparison
    ax3 = plt.subplot(3, 4, 3)
    valid_mag = data['valid_mag']
    if np.sum(valid_mag) > 0:
        x_mag = data['ab_mag_personal'][valid_mag]
        y_mag = data['mag_public'][valid_mag]
        
        ax3.scatter(x_mag, y_mag, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_mag), np.min(y_mag))
        max_val = max(np.max(x_mag), np.max(y_mag))
        ax3.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax3.set_xlabel('Personal Catalogue AB Mag')
        ax3.set_ylabel('Public Catalogue AB Mag')
        ax3.set_title('AB Magnitude Comparison (F770W)')
        ax3.grid(True, alpha=0.3)
        
        # Add statistics
        stats_mag = calculate_statistics(x_mag, y_mag, np.ones(len(x_mag), dtype=bool))
        if stats_mag:
            ax3.text(0.05, 0.95, f'r = {stats_mag["correlation"]:.3f}\nN = {stats_mag["n_objects"]}', 
                    transform=ax3.transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 4. Signal-to-noise ratio comparison
    ax4 = plt.subplot(3, 4, 4)
    valid_snr = data['valid_flux'] & data['valid_flux_err']
    if np.sum(valid_snr) > 0:
        snr_personal = data['flux_personal'][valid_snr] / data['flux_err_personal'][valid_snr]
        snr_public = data['flux_public'][valid_snr] / data['flux_err_public'][valid_snr]
        
        ax4.scatter(snr_personal, snr_public, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(snr_personal), np.min(snr_public))
        max_val = max(np.max(snr_personal), np.max(snr_public))
        ax4.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax4.set_xlabel('Personal Catalogue S/N')
        ax4.set_ylabel('Public Catalogue S/N')
        ax4.set_title('Signal-to-Noise Ratio Comparison')
        ax4.grid(True, alpha=0.3)
        
        # Add statistics
        stats_snr = calculate_statistics(snr_personal, snr_public, np.ones(len(snr_personal), dtype=bool))
        if stats_snr:
            ax4.text(0.05, 0.95, f'r = {stats_snr["correlation"]:.3f}\nN = {stats_snr["n_objects"]}', 
                    transform=ax4.transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 5. Flux residuals vs flux
    ax5 = plt.subplot(3, 4, 5)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        residuals = y_flux - x_flux
        
        ax5.scatter(x_flux, residuals, alpha=0.6, s=30, color=colors['scatter'])
        ax5.axhline(y=0, color=colors['line'], linestyle='--', lw=2)
        ax5.set_xscale('log')
        ax5.set_xlabel('Personal Catalogue Flux (µJy)')
        ax5.set_ylabel('Flux Residuals (Public - Personal)')
        ax5.set_title('Flux Residuals vs Flux')
        ax5.grid(True, alpha=0.3)
    
    # 6. Fractional flux differences
    ax6 = plt.subplot(3, 4, 6)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        frac_diff = (y_flux - x_flux) / x_flux
        
        ax6.scatter(x_flux, frac_diff, alpha=0.6, s=30, color=colors['scatter'])
        ax6.axhline(y=0, color=colors['line'], linestyle='--', lw=2)
        ax6.set_xscale('log')
        ax6.set_xlabel('Personal Catalogue Flux (µJy)')
        ax6.set_ylabel('Fractional Flux Difference')
        ax6.set_title('Fractional Flux Differences')
        ax6.grid(True, alpha=0.3)
    
    # 7. Magnitude residuals vs magnitude
    ax7 = plt.subplot(3, 4, 7)
    if np.sum(valid_mag) > 0:
        x_mag = data['ab_mag_personal'][valid_mag]
        y_mag = data['mag_public'][valid_mag]
        mag_residuals = y_mag - x_mag
        
        ax7.scatter(x_mag, mag_residuals, alpha=0.6, s=30, color=colors['scatter'])
        ax7.axhline(y=0, color=colors['line'], linestyle='--', lw=2)
        ax7.set_xlabel('Personal Catalogue AB Mag')
        ax7.set_ylabel('Magnitude Residuals (Public - Personal)')
        ax7.set_title('Magnitude Residuals vs Magnitude')
        ax7.grid(True, alpha=0.3)
    
    # 8. Error bar comparison
    ax8 = plt.subplot(3, 4, 8)
    if np.sum(valid_flux) & np.sum(valid_flux_err) > 0:
        valid_both = valid_flux & valid_flux_err
        x_flux = data['flux_personal'][valid_both]
        y_flux = data['flux_public'][valid_both]
        x_err = data['flux_err_personal'][valid_both]
        y_err = data['flux_err_public'][valid_both]
        
        # Plot a subset of points with error bars to avoid cluttering
        n_plot = min(50, len(x_flux))
        indices = np.random.choice(len(x_flux), n_plot, replace=False)
        
        ax8.errorbar(x_flux[indices], y_flux[indices], 
                    xerr=x_err[indices], yerr=y_err[indices],
                    fmt='o', alpha=0.6, capsize=3, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_flux), np.min(y_flux))
        max_val = max(np.max(x_flux), np.max(y_flux))
        ax8.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax8.set_xscale('log')
        ax8.set_yscale('log')
        ax8.set_xlabel('Personal Catalogue Flux (µJy)')
        ax8.set_ylabel('Public Catalogue Flux (µJy)')
        ax8.set_title(f'Flux with Error Bars (N={n_plot} random subset)')
        ax8.grid(True, alpha=0.3)
    
    # 9. Flux histogram comparison
    ax9 = plt.subplot(3, 4, 9)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        
        bins = np.logspace(np.log10(min(np.min(x_flux), np.min(y_flux))), 
                          np.log10(max(np.max(x_flux), np.max(y_flux))), 20)
        
        ax9.hist(x_flux, bins=bins, alpha=0.6, label='Personal', color=colors['scatter'])
        ax9.hist(y_flux, bins=bins, alpha=0.6, label='Public', color=colors['line'])
        ax9.set_xscale('log')
        ax9.set_xlabel('Flux (µJy)')
        ax9.set_ylabel('Number of Objects')
        ax9.set_title('Flux Distribution Comparison')
        ax9.legend()
        ax9.grid(True, alpha=0.3)
    
    # 10. Magnitude histogram comparison
    ax10 = plt.subplot(3, 4, 10)
    if np.sum(valid_mag) > 0:
        x_mag = data['ab_mag_personal'][valid_mag]
        y_mag = data['mag_public'][valid_mag]
        
        bins = np.linspace(min(np.min(x_mag), np.min(y_mag)), 
                          max(np.max(x_mag), np.max(y_mag)), 20)
        
        ax10.hist(x_mag, bins=bins, alpha=0.6, label='Personal', color=colors['scatter'])
        ax10.hist(y_mag, bins=bins, alpha=0.6, label='Public', color=colors['line'])
        ax10.set_xlabel('AB Magnitude')
        ax10.set_ylabel('Number of Objects')
        ax10.set_title('Magnitude Distribution Comparison')
        ax10.legend()
        ax10.grid(True, alpha=0.3)
    
    # 11. Residuals histogram
    ax11 = plt.subplot(3, 4, 11)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        frac_diff = (y_flux - x_flux) / x_flux
        
        ax11.hist(frac_diff, bins=30, alpha=0.7, color=colors['hist'])
        ax11.axvline(x=0, color=colors['line'], linestyle='--', lw=2)
        ax11.axvline(x=np.median(frac_diff), color='red', linestyle='-', lw=2, label=f'Median: {np.median(frac_diff):.3f}')
        ax11.set_xlabel('Fractional Flux Difference')
        ax11.set_ylabel('Number of Objects')
        ax11.set_title('Fractional Difference Distribution')
        ax11.legend()
        ax11.grid(True, alpha=0.3)
    
    # 12. Summary statistics text
    ax12 = plt.subplot(3, 4, 12)
    ax12.axis('off')
    
    summary_text = "PHOTOMETRY COMPARISON SUMMARY\n\n"
    
    if np.sum(valid_flux) > 0:
        stats_flux = calculate_statistics(data['flux_personal'][valid_flux], 
                                        data['flux_public'][valid_flux], 
                                        np.ones(np.sum(valid_flux), dtype=bool))
        if stats_flux:
            summary_text += f"FLUX COMPARISON (N={stats_flux['n_objects']}):\n"
            summary_text += f"  Correlation: {stats_flux['correlation']:.3f}\n"
            summary_text += f"  Mean fractional diff: {stats_flux['mean_frac_diff']:.3f}\n"
            summary_text += f"  Std fractional diff: {stats_flux['std_frac_diff']:.3f}\n\n"
    
    if np.sum(valid_mag) > 0:
        stats_mag = calculate_statistics(data['ab_mag_personal'][valid_mag], 
                                       data['mag_public'][valid_mag], 
                                       np.ones(np.sum(valid_mag), dtype=bool))
        if stats_mag:
            summary_text += f"MAGNITUDE COMPARISON (N={stats_mag['n_objects']}):\n"
            summary_text += f"  Correlation: {stats_mag['correlation']:.3f}\n"
            summary_text += f"  Mean residual: {stats_mag['mean_residual']:.3f} mag\n"
            summary_text += f"  RMS residual: {stats_mag['rms_residual']:.3f} mag\n\n"
    
    summary_text += f"DATA COVERAGE:\n"
    summary_text += f"  Total objects: {len(data['table'])}\n"
    summary_text += f"  Valid flux measurements: {np.sum(valid_flux)}\n"
    summary_text += f"  Valid flux errors: {np.sum(valid_flux_err)}\n"
    summary_text += f"  Valid magnitudes: {np.sum(valid_mag)}\n"
    
    ax12.text(0.05, 0.95, summary_text, transform=ax12.transAxes, 
             verticalalignment='top', fontfamily='monospace', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    #plt.suptitle('MIRI F770W Photometry Comparison: Personal vs Public Catalogue', 
    #            fontsize=24, y=0.98)
    
    return fig

# Main execution function
def analyze_photometry_comparison(fits_file_path, output_plot_path=None):
    """
    Main function to perform photometry comparison analysis
    """
    print("Loading and preparing data...")
    data = load_and_prepare_data(fits_file_path)
    
    print("Creating comparison plots...")
    fig = create_comparison_plot(data)
    
    if output_plot_path:
        print(f"Saving plot to {output_plot_path}")
        plt.savefig(output_plot_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print("PHOTOMETRY COMPARISON ANALYSIS SUMMARY")
    print("="*60)
    
    valid_flux = data['valid_flux']
    valid_mag = data['valid_mag']
    
    if np.sum(valid_flux) > 0:
        stats_flux = calculate_statistics(data['flux_personal'][valid_flux], 
                                        data['flux_public'][valid_flux], 
                                        np.ones(np.sum(valid_flux), dtype=bool))
        if stats_flux:
            print(f"\nFLUX COMPARISON ({stats_flux['n_objects']} objects):")
            print(f"  Pearson correlation coefficient: {stats_flux['correlation']:.4f}")
            print(f"  Mean fractional difference: {stats_flux['mean_frac_diff']:.4f}")
            print(f"  Standard deviation of fractional differences: {stats_flux['std_frac_diff']:.4f}")
            print(f"  RMS of absolute residuals: {stats_flux['rms_residual']:.4f} µJy")
    
    if np.sum(valid_mag) > 0:
        stats_mag = calculate_statistics(data['ab_mag_personal'][valid_mag], 
                                       data['mag_public'][valid_mag], 
                                       np.ones(np.sum(valid_mag), dtype=bool))
        if stats_mag:
            print(f"\nMAGNITUDE COMPARISON ({stats_mag['n_objects']} objects):")
            print(f"  Pearson correlation coefficient: {stats_mag['correlation']:.4f}")
            print(f"  Mean magnitude residual: {stats_mag['mean_residual']:.4f} mag")
            print(f"  Standard deviation of residuals: {stats_mag['std_residual']:.4f} mag")
            print(f"  RMS of residuals: {stats_mag['rms_residual']:.4f} mag")
    
    print(f"\nDATA COVERAGE:")
    print(f"  Total objects in table: {len(data['table'])}")
    print(f"  Objects with valid flux measurements: {np.sum(valid_flux)}")
    print(f"  Objects with valid magnitude measurements: {np.sum(valid_mag)}")
    
    return data, fig


In [ ]:
 # Replace with your FITS file path
fits_file_path = '/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_id_matched.fits'


# Optional: specify output path for the plot
output_plot_path = '/home/bpc/University/master/Red_Cardinal/photometry/photometry_comparisons/photometry_comparison_plot.png'

# Run the analysis
data, fig = analyze_photometry_comparison(fits_file_path, output_plot_path)